# Automatically detecting strokes and fills

A lot of times, for whatever reason, the strokes and fills are separated out. I think the stroke and fill for a shape can be easily detected from the geometry and I'm vary of asking annotators to give us this data.

In [ ]:
from vectorrvnn.utils import *
from vectorrvnn.geometry import *
from vectorrvnn.data import *
from tqdm import tqdm
from scipy.spatial import KDTree
    
svgfiles = list(filter(lambda x : x.endswith('svg'), allfiles('../Emojis')))[100:101]
trees = [SVGData(f) for f in svgfiles]
for tree in tqdm(trees): 
    plt.imshow(rasterize(tree.doc, 200, 200))
    plt.show()
    for g in maximalCliques(tree, [fourier_descriptor, centroid], 1e-2):
        plt.imshow(rasterize(subsetSvg(tree.doc, g), 100, 100))
        plt.show()


The first cut worked well when the stroke and fills completely matched. It didn't work out in cases such as the one above. The two shapes are only slightly different. Fourier descriptor doesn't think so. So now I'm going to try shape contexts.

In [ ]:
@lru_cache(maxsize=100)
def shapeContexts (doc, i, nSamples=100) : 
    """ Give a list of shape contexts that can be used for matching """
    path = cachedPaths(doc)[i].path
    pts = np.array(equiDistantSamples(doc, path, nSamples, normalize=True)).T
    grid = pts.reshape((-1, 1, 2))
    grid = np.repeat(grid, nSamples, axis=1)
    diffs = grid - pts
    mask = np.eye(nSamples).astype(bool)
    diffs = diffs[~mask].reshape(nSamples, nSamples - 1, 2)  
    logNorms = np.log2(np.linalg.norm(diffs, axis=2) + 1e-7)
    thetas = np.arctan2(diffs[:, :, 0], diffs[:, :, 1])
    xbins = np.linspace(-10, 0.5, 10)
    ybins = np.linspace(-np.pi, np.pi, 10)
    contexts = []
    # Figure out how to vectorize this step.
    for i in range(nSamples) : 
        H, *_ = np.histogram2d(logNorms[i], thetas[i], bins=[xbins, ybins], density=True)
        contexts.append(H)
    return contexts

@lru_cache(maxsize=100)
def error2 (doc, i, j, nSamples=100) :
    def chi2 (c1, c2) : 
        return (0.5 * (c1 - c2) ** 2 / (c1 + c2 + 1e-5)).sum()
    cn1, cn2 = centroid(doc, i), centroid(doc, j)
    ctx1 = shapeContexts(doc, i, nSamples)
    ctx2 = shapeContexts(doc, j, nSamples) 
    costDict = dict()
    for (i, ci), (j, cj) in product(enumerate(ctx1), enumerate(ctx2)) : 
        costDict[(i, j)] = chi2(ci, cj)
    matching = optimalBipartiteMatching(costDict)
    costs = []
    for i, j in matching.items() : 
        costs.append(costDict[(i, j)])
    err = np.median(costs) + np.linalg.norm(cn1 - cn2)
    return err

# for g in maximalCliques(doc, error2, 2e-1): 
#     print(g)
#     plt.imshow(rasterize(subsetSvg(doc, g), 100, 100))
#     plt.show()